In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres

# Load Data

In [3]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
def read_data(pair):
    db = os.path.join(dropbox_dir,'sqlite', 'ploniex-chart-data', f'{pair}.db')
    data = load_trading_data(db, 'chart_data', from_date='2018-01-01', period='30min')
    data['ohlc4'] = (data['open'] + data['close'] + data['high'] + data['low']) / 4
    return data

In [4]:
train_pairs = pd.read_csv('pairs_train.tsv', header=None)[0].tolist()
all_pairs = pd.read_csv('pairs.tsv', header=None)[0].tolist()
data_dict = {pair: read_data(pair) for pair in all_pairs}

In [5]:
rpath = os.path.join(dropbox_dir,'result', 'base-strategy-learn-03.tsv')
results = read_tsv(rpath)

In [6]:
results.shape

(210000, 17)

In [7]:
results.head()

,pair,window_size,target,stop_loss,_return,startAmount,numberOfTrades,totalProfit,avgProfit,winPercentage,avgWinTrade,avgLossTrade,profitFactor,maxDrawdown,avgTimeToClose,minTimeToClose,maxTimeToClose
0,BTC_BAT,4,0.0100,0.0100,-0.1000,100.0000,1.0000,-1.4680,-1.4680,0.0000,0.0000,-1.4680,0.0000,1.4680,0.0000,0.0000,0.0000
1,BTC_BAT,4,0.0100,0.0100,-0.0300,100.0000,58.0000,-18.3624,-0.3166,46.5517,0.8953,-1.3721,0.5683,19.4791,2.2241,0.0000,12.0000
2,BTC_BAT,4,0.0100,0.0100,-0.0200,100.0000,101.0000,-20.7168,-0.2051,50.4950,0.8786,-1.3106,0.6838,23.3164,4.6931,0.0000,47.0000
3,BTC_BAT,4,0.0100,0.0100,-0.0400,100.0000,32.0000,-16.0294,-0.5009,40.6250,0.9099,-1.4662,0.4246,16.8469,1.9062,0.0000,8.0000
4,BTC_BAT,4,0.0100,0.0100,-0.0600,100.0000,12.0000,-12.5527,-1.0461,25.0000,0.9336,-1.7060,0.1824,14.2470,1.6667,0.0000,7.0000


# Pairs Volume

In [8]:
volume_stats = pd.DataFrame()
for pair in all_pairs:
    data = data_dict[pair]
    pair_stats = data[['volume']].describe().transpose()
    pair_stats['sum'] = data['volume'].sum()
    pair_stats['pair'] = pair
    volume_stats = volume_stats.append(pair_stats)

cols = list(volume_stats.columns)
cols = cols[-1:] + cols[:-1]
volume_stats = reset_index_hard(volume_stats).sort_values('mean', ascending=False)[cols]

In [9]:
volume_stats_train = volume_stats[volume_stats['pair'].isin(train_pairs)]

# Find best params

In [11]:
def agg(results):
    cols = ['window_size', 'target', 'stop_loss', '_return']
    aggregated = results.groupby(cols)\
        .agg({
            'winPercentage': ['min', 'mean'],
            'maxTimeToClose': 'max',
            'avgTimeToClose': 'mean',
            'numberOfTrades': 'mean',
            'totalProfit': ['mean', 'min', 'max', 'median']
        })\
        .reset_index()
    aggregated.columns = ['_'.join(cols) if cols[1] != '' else cols[0] for cols in aggregated.columns]
    return aggregated

aggregated = agg(results)

top_volume_pairs = volume_stats_train.head(10)['pair'].tolist()
aggregated_top = agg(results[results['pair'].isin(top_volume_pairs)])

### All coins

In [20]:
DAY = 48
def sort_params(aggregated):
    return aggregated\
        .query(f'totalProfit_min >= 0')\
        .query(f'maxTimeToClose_max < {10*DAY}')\
        .query(f'winPercentage_min >= 70')\
        .sort_values(['totalProfit_mean', 'winPercentage_mean'], ascending=False)

In [24]:
sort_params(aggregated).head(10)

,window_size,target,stop_loss,_return,winPercentage_min,winPercentage_mean,maxTimeToClose_max,avgTimeToClose_mean,numberOfTrades_mean,totalProfit_mean,totalProfit_min,totalProfit_max,totalProfit_median
3058,12,0.0100,0.0600,-0.0200,88.0952,95.1375,395.0000,11.1833,92.5000,82.2445,0.3556,169.6319,92.9861
4089,24,0.0100,0.0900,-0.0100,92.3077,98.3564,468.0000,14.1862,53.1333,53.5509,1.7893,98.3353,58.0361
4069,24,0.0100,0.0700,-0.0100,90.5660,97.8154,468.0000,13.5033,53.4333,52.6510,4.0403,98.3353,55.9971
4186,24,0.0200,0.0900,-0.0400,84.8485,95.2442,477.0000,28.7042,30.6000,51.1126,1.1885,126.4860,46.9698
4059,24,0.0100,0.0600,-0.0100,88.8889,96.8053,468.0000,12.8010,53.5667,50.9833,1.2909,92.5991,57.6612
4166,24,0.0200,0.0700,-0.0400,81.8182,92.8250,477.0000,24.3388,31.1000,48.7121,3.0945,111.1410,48.4102
4156,24,0.0200,0.0600,-0.0400,80.0000,90.9800,477.0000,23.0189,31.2333,46.2440,1.2500,91.1397,40.7036
4146,24,0.0200,0.0500,-0.0400,78.5714,89.1703,477.0000,21.5979,31.3000,42.4451,1.2500,96.3986,42.8248
4185,24,0.0200,0.0900,-0.0500,86.6667,95.5339,477.0000,24.7712,26.1333,42.1266,7.3952,103.0794,41.7471
4088,24,0.0100,0.0900,-0.0200,93.0233,98.3017,468.0000,13.5036,43.4333,41.5666,3.4252,81.6196,43.4514


### Top Coins

In [23]:
sort_params(aggregated_top).head(10)

,window_size,target,stop_loss,_return,winPercentage_min,winPercentage_mean,maxTimeToClose_max,avgTimeToClose_mean,numberOfTrades_mean,totalProfit_mean,totalProfit_min,totalProfit_max,totalProfit_median
3058,12,0.0100,0.0600,-0.0200,88.8889,94.3137,378.0000,13.8285,81.9000,63.9069,0.3556,133.0973,71.7525
3038,12,0.0100,0.0400,-0.0200,85.3333,91.2120,378.0000,12.1481,83.0000,55.3747,6.0259,107.0614,60.8866
2147,8,0.0200,0.0500,-0.0300,78.1250,83.5365,426.0000,24.0850,67.4000,50.0514,0.7099,134.3859,35.5298
4049,24,0.0100,0.0500,-0.0100,88.8889,96.0931,355.0000,17.0437,54.3000,47.2515,1.5993,86.4640,48.0672
4059,24,0.0100,0.0600,-0.0100,90.7407,96.4104,355.0000,17.3448,54.2000,46.6605,6.6735,84.4460,43.1075
4069,24,0.0100,0.0700,-0.0100,90.5660,96.9370,396.0000,18.0886,53.9000,46.0020,4.0403,84.4460,41.5631
4089,24,0.0100,0.0900,-0.0100,92.3077,97.6666,447.0000,19.5677,53.0000,45.0662,1.7893,78.0483,47.8739
2057,8,0.0100,0.0600,-0.0300,90.2778,93.8363,336.0000,10.4930,67.6000,44.8182,8.2953,120.3309,30.3564
1057,6,0.0100,0.0600,-0.0300,90.7692,93.7828,352.0000,8.7778,74.4000,43.9287,6.3702,101.7175,41.0721
2047,8,0.0100,0.0500,-0.0300,90.0000,93.0242,336.0000,9.2477,68.1000,41.6433,10.3747,112.7259,32.7278
